### Developer: Mani kanta

### Aim: Embedding generation using Skip Gram Model

# Importing Libraries

In [13]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import zipfile
import collections
import math
import random

from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as Func
from torch.optim.lr_scheduler import StepLR
import time

from s3fs.core import S3FileSystem
import io
s3 = S3FileSystem()


In [14]:
ls

embedding_1.pth
Embedding_Model_Aug_Data_Batch.ipynb
Embedding_Model_Full_Data_Batch_GPU.ipynb
Embedding_Model_Full_Data_Batch.ipynb
Embedding_Model_Full_Data.ipynb
Embedding_Model.ipynb
Embedding Preprocessing.ipynb
embedding_v1.pth
embedding_v2.pth
Hotel.csv
idx2word_aug.npy
neg_sample_city_aug.npy
neg_sample_country_aug.npy
part-00000-34fc8918-9837-47bb-9d5e-c1babbd238a5-c000.snappy.parquet
pos_sample_aug.npy
train_data.npy
T-sne plot_1.ipynb
T-sne plot_v1.ipynb
T-sne plot_v2.ipynb
within_city.npy
within_country.npy


In [15]:
with open('pos_sample_aug.npy', 'rb') as f:
    train_data = np.load(f,allow_pickle=True )

In [16]:
with open('neg_sample_city_aug.npy', 'rb') as f:
    within_city = np.load(f,allow_pickle=True )

In [17]:
with open('neg_sample_country_aug.npy', 'rb') as f:
    within_country = np.load(f,allow_pickle=True )

In [18]:
len(train_data)

6950523

In [19]:
len(within_city)

6950523

In [20]:
len(within_country)

6950523

In [21]:
import pandas as pd
hotel = pd.read_csv("Hotel.csv")
hotel.shape

(81417, 5)

In [22]:
hotel

,Unnamed: 0,hotel_id,city_id,city_name,country_id
0,0,94319,678.0,Nha-Trang,217
1,1,13195,4.0,Bangalore,1
2,2,89191,4.0,Bangalore,1
3,3,86999,421.0,Karimnagar,1
4,4,87204,677.0,Krabi,8
...,...,...,...,...,...
81412,81412,62801,1.0,Gurgaon,1
81413,81413,97059,14.0,Kolkata,1
81414,81414,110181,1.0,Gurgaon,1
81415,81415,5298,5.0,Mumbai,1


In [23]:
class skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(skipgram, self).__init__()
        self.u_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)   
        self.v_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True) 
        self.embedding_dim = embedding_dim
        self.init_emb()
    
    def init_emb(self):
        initrange = 0.5 / self.embedding_dim
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
    
    def forward(self, u_pos, v_pos, v_neg_city, v_neg_country):

        embed_u = self.u_embeddings(u_pos)
        embed_v = self.v_embeddings(v_pos)
        
        embed_u = embed_u.unsqueeze(0)

        score  = torch.mul(embed_u, embed_v)
        score = torch.sum(score, dim=1)
        log_target = F.logsigmoid(score).squeeze()

        neg_embed_v_city = self.v_embeddings(v_neg_city)
        neg_embed_v_country = self.v_embeddings(v_neg_country)

        neg_score_city = torch.mul(neg_embed_v_city, embed_u)
        neg_score_city = torch.sum(neg_score_city, dim=1)
        sum_log_neg_score_city = F.logsigmoid(-1*neg_score_city).squeeze()
        
        neg_score_country = torch.mul(neg_embed_v_country, embed_u)
        neg_score_country = torch.sum(neg_score_country, dim=1)
        sum_log_neg_score_country = F.logsigmoid(-1*neg_score_country).squeeze()

        loss = log_target.sum() + sum_log_neg_score_city.sum() + sum_log_neg_score_country.sum()

        return -1*loss
    
    def input_embeddings(self):
        return self.u_embeddings.weight.data.cpu().numpy()
   
    def save_model(model, model_dir):
        logger.info("Saving the model.")
        path = os.path.join(model_dir, "embedding_aug_BGD.pth")
        torch.save(model.cpu().state_dict(), path)


In [ ]:
model = skipgram(81417, 200)
batch_size = 1
optimizer = optim.SGD(model.parameters(),lr=0.2)
start = time.time()
print(start)
for epoch in range(100):
    epoch_start = time.time()     
    batch_num = 0
    batch_new = 0
    epoch_loss = 0
    
    for i in tqdm(range(len(train_data)//batch_size)):
        loss = 0
        for j in range(batch_size):
            center_id = torch.tensor(train_data[i+j][0])
            positive = torch.tensor(train_data[i+j][1])
            neg_within_city = torch.tensor(within_city[i+j])
            neg_within_country = torch.tensor(within_country[i+j])
            optimizer.zero_grad()
            loss = loss + model(center_id, positive, neg_within_city, neg_within_country)
            # print(loss)
        loss = loss/(batch_size*1.0)
        epoch_loss = epoch_loss + loss
        loss.backward()
        optimizer.step()
        batch_num = batch_num + 1         
            
#     print("epoch done!! - ",epoch ," time taken: ", time.time() - epoch_start)
    print(epoch_loss/(len(train_data)))
            
print("Optimization Finished!")
print("total time: ", time.time()- start)

  0%|          | 0/6950523 [00:00<?, ?it/s]

1635827719.5412352


 11%|█         | 773439/6950523 [13:59<1:51:18, 924.89it/s] 

In [17]:
with open('idx2word_aug.npy', 'rb') as f:
    idx2word = np.load(f,allow_pickle=True )

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
167115/143509

In [ ]:
0.01